In [3]:
from src.tools_dev import Mapdisplay, random_point_in_shp
import ee
import tensorflow as tf
import geopandas as gpd
from shapely.geometry import Point, Polygon
from folium.plugins import MarkerCluster
import numpy as np
from src.tools_dev import get_bound_points, generate_box_buffer, \
                            generate_ee_points, get_landsat_data
import geopy.distance
import rasterio
import matplotlib.pyplot as plt
import pandas as pd

import folium
print('Folium version: ' + folium.__version__)

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'
ee.Initialize()
%load_ext autoreload
%autoreload 2
%matplotlib inline

Folium version: 0.11.0
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
def build_features(shpfile):
    df = gpd.read_file(shp).to_crs("EPSG:4326")
    df = df[df.area==np.max(df.area)].reset_index()
    xmin, ymin, xmax, ymax = df.bounds.values[0]

    x,y = df.geometry[0].exterior.coords.xy
    cords = np.dstack((x,y)).tolist()
    g = ee.Geometry.Polygon(cords)
    feature = ee.Feature(g)
    fc = ee.FeatureCollection([feature])
    return(fc,g)

def normalize(img):
    return (img-np.nanmin(img))/(np.nanmax(img)-(np.nanmin(img)))

def clipper(img):
    return img.clip(fc)
  
def addTime(image): 
    constant = 1000 * 60 * 60 * 24 * 365
    return image.addBands(image.metadata('system:time_start').divide(constant))

def fill(image):
    filled1a = image.focal_mean(2, 'square', 'pixels', 1).blend(image)
    return filled1a.set('system:time_start',image.get('system:time_start'))

def climate_collection_generator(bounds):
    
    tc = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterBounds(bounds)
       
    return tc 

def image_collection_export(ic, outbands, BUCKET, PREFIX, DESC, REGION):
    task = ee.batch.Export.image.toCloudStorage(\
                                         image = ic,\
                                         description = DESC,\
                                         bucket = BUCKET,\
                                         fileNamePrefix = PREFIX,\
                                         scale = 30,\
                                         region = REGION)
    task.start()    
    return 

def ic_annual_aggregate(ic, outbands, years, month_range):
    out_images = []
    for year in years:
        annual_image_group = []
        #renamed = [f'{b}_{year}' for b in outbands]
        for b in outbands:
            renamed = f'{b}_{year}'
            if b == 'pr':
                image_annual = ic.filter(ee.Filter.calendarRange(int(year),int(year),'year'))\
                    .filter(ee.Filter.calendarRange(int(month_range[0]),int(month_range[1]),'month'))\
                    .select([b])\
                    .sum()\
                    .rename([renamed]).float()
            else:
                image_annual = ic.filter(ee.Filter.calendarRange(int(year),int(year),'year'))\
                    .filter(ee.Filter.calendarRange(int(month_range[0]),int(month_range[1]),'month'))\
                    .select([b])\
                    .mean()\
                    .rename([renamed]).float()
            annual_image_group.append(image_annual)
        out_images.append(ee.Image.cat(annual_image_group))
    out = ee.Image.cat(out_images)    
    return out
    

In [43]:
BUCKETS = ['ikh-nart','rio-mora']

for BUCKET in BUCKETS:
    if BUCKET == 'rio-mora':
        shp = './data/shp/RioMoraNWRBndry_2019/RioMoraNWRBndry_2019.shp'
        target_months = (4,10) 
        #shp = './data/shp/MRW/Mora River Watershed July2011.shp'
    elif BUCKET == 'ikh-nart':
        shp = './data/shp/IN_boundary/IN_boundary.shp'
        target_months = (6,10) 

    #need an automated way of determining growing season....
    fc,g = build_features(shp)
    REGION = g.getInfo()['coordinates']

    years = np.arange(1986,2021) 
    # Define a FeatureCollection fc above
    #Build landsat collection
    outbands = ['pr', 'tmmx', 'tmmn']
    terra = climate_collection_generator(bounds=fc) 
    terra = terra.select(outbands).map(clipper)
    #landsat = landsat.map(clipper)
    
    month_groups = (1,3),(4,6),(7,9),(10,12)
    groups = ['JFM', 'AMJ', 'JAS', 'OND']
    
    #image = ic_annual_aggregate(terra, outbands, years, month_groups[0]) 
    ######## ALL BANDS #############
    for i,m in enumerate(month_groups):
        DESC = f'{BUCKET}__{groups[i]}'
        PREFIX = f'processed/terra_{groups[i]}'
        out = ic_annual_aggregate(terra, outbands, years, target_months)
        #export these to bucket
        image_collection_export(out, outbands, BUCKET, PREFIX, DESC, REGION)
        del out

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until
